# Clarity - Interactive Tour

*Clarity is a **decidable** smart contract language that optimizes for predictability and security, designed by Blockstack. Smart contracts allow developers to encode essential business logic on a blockchain.*

*This notebook is part of a suite of noteboooks, designed for helping developpers getting started with Clarity.*


***
**This notebook was designed for encouraging experimentation. Feel free to execute the snippets, tweak values, refactor with your own Clarity code.**
***


# Chapter 2 - Data types

In this chapter, we will gradually learn about the data types powering Clarity, from the most basic ones, to the most sophisticated. This chapter will gives us the fundamental keys to approach the next chapters.

## 1) Primitive types

Let's look at a first set of types natively available.


### bool

The bool type is a datatype which can be either `true` or `false`. It is used in comparisons and bitwise operations like `if`, `and`, `not`, `or`, etc.

In [63]:
(print (is-eq true false))

false

***
### integer

The signed integer types are encoded on 128 bits. As a result, minimum is -(2^127), and max is 2^127-1.
Both overflow and underflows emit exceptions and abort execution.

In [64]:
(print 1)

1

***
### unsigned integer

The unsigned integer types are encoded on 128 bits. As a result, minimum is 0, and max is 2^128-1.
Both overflow and underflows emit exceptions and abort execution.

In [65]:
(print u1) 

u1

***
### buffer

The buffer type is used to represent a fixed-length sequence of bytes.  

In [66]:
(print 0x0123456789abcdef)

0x0123456789abcdef

***
### ascii string

The ascii string type is used to represent a fixed sequence of printable ASCII characters. This includes:

- ASCII Alphanumeric characters
- Whitespace characters: `'\t' '\n' '\x0C' '\r' ' '`
- Punctuation: `! " # $ % & ' ( ) * + , - . / : ; < = > ? @ [ \ ] ^ _{ | } ~`


In [67]:
(print "Hello world") 

"Hello world"

***
### utf8 string

The utf8 string type is used to represent a fixed sequence of printable UTF8 characters. More specifically, since 'character' isn't a well-defined concept in Unicode, char is a 'Unicode scalar value'.


In [82]:
(print (len u"\u{1F436}")) ;; the len of a fox emoticon is 1

u1

***
### principal

The principal type is used to represent STX address.

## 2) Declaring constants

When working with Clarity, developers are working with immutable data. 
Once bound to a name, a value won't and can't be mutated.
We will see in the next chapter that some functions can be used for persisting and saving states.

### (define-constant name expression)

`define-constant` is used to define a private constant value in a smart contract.
The expression passed into the definition is evaluated at contract launch, in the order that it is
supplied in the contract. This can lead to undefined function or undefined variable errors in the
event that a function or variable used in the expression has not been defined before the constant.

Like other kinds of definition statements, `define-constant` may only be used at the top level of a smart contract
definition (i.e., you cannot put a define statement in the middle of a function body).

In [13]:
(define-constant my-constant 4)

(print my-constant) ;; returns 4

4

***
### (let ((name1 expr1) (name2 expr2) ...) expr-body1 expr-body2 ... expr-body-last)
The `let` function accepts a list of `variable name` and `expression` pairs,
evaluating each expression and _binding_ it to the corresponding variable name. The _context_
created by this set of bindings is used for evaluating its body expressions. The let expression returns the value of the last such body expression.

The `let` construct presents some complexity at first sight, however it is the backbone for contracts in Clarity. Stay put, and let's look at a few instanciations and usages of this function.

In [3]:
(let ((str1 "Hello"))
    (print str1))

"Hello"

In [5]:
(let ((str1 "Hello")
      (str2 " world"))
    (print (concat str1 str2)))

"Hello world"

In [7]:
(let ((str1 "Hello")
      (str2 " world"))
    (asserts! (is-eq str1 "Hello") "Error")
    (asserts! (is-eq str2 " world") "Error")
    (print (concat str1 str2)))

"Hello world"

In [8]:
(let ((str1 "Hello")
      (str2 " world"))
    (let ((str3 (concat str1 str2)))
        (print str3)))

"Hello world"

todo(ludo): conclusion

## 3) Compositioning types into structures

Programmers can structure data types using tuple constructs.
The syntax is the following:

In [16]:
(print {
    first-name: "satoshi", 
    last-name: "nakamoto"
})

(tuple (first-name "satoshi") (last-name "nakamoto"))

This compositions can be nested, for creating complex structures:

In [18]:
(print {
    user: {
        first-name: "satoshi", 
        last-name: "nakamoto"
    },
    wallet: {
        address: 'SPAXYA5XS51713FDTQ8H94EJ4V579CXMTRNBZKSF
    }
})

(tuple (person (tuple (first-name "satoshi") (last-name "nakamoto"))) (wallet (tuple (address SPAXYA5XS51713FDTQ8H94EJ4V579CXMTRNBZKSF))))

It's important to note that these structures are considered as any other types, implying that they can be instancianted with the constructs previously explained.

In [20]:
(define-constant satoshi { first-name: "satoshi", last-name: "nakamoto" })

(print satoshi)

(tuple (first-name "satoshi") (last-name "nakamoto"))

Specific fields can be extracted and read from this structures, using the function `get`:

### (get key-name tuple)

The `get` function fetches the value associated with a given key from the supplied typed tuple.
If an `Optional` value is supplied as the inputted tuple, `get` returns an `Optional` type of the specified key in
the tuple.

In [27]:
(let ((user { first-name: "satoshi", last-name: "nakamoto" }))
    (print (get first-name user)))

"satoshi"

## 4) Aggregating types in lists

In this section, we will introduce lists at a basic level - this subject will be covered in depth in a dedicated notebook.
In order to guarantee termination, lists, in Clarity are constrained and have a static size.

The syntax is the following:

In [4]:
(define-constant my-list (list "alice" "bob" "charlie"))

(print my-list)

("alice" "bob" "charlie")

Lists can be nested to construct multi-dimensional vectors:

In [10]:
(define-constant my-matrix (list (list 1 0 0) (list 0 1 0) (list 0 0 1)))

(print my-matrix)

((1 0 0) (0 1 0) (0 0 1))

Finally, and as you would expect, tuple-based lists can also be constructed:

In [84]:
(define-constant user-1 { first-name: "satoshi", last-name: "nakamoto" })
(define-constant user-2 { first-name: "john", last-name: "doe" })

(print (list user-1 user-2))

((tuple (first-name "satoshi") (last-name "nakamoto")) (tuple (first-name "john") (last-name "doe")))

## 5) The optional type and its associated functions

The type `option` represents an optional value: every `option` is either Some and contains a value, or None, and does not.
The type Option is an alternative to the concept of `nullable` values in other languages, and was picked over `Null` for the discipline it enforces on the programmation style.

Let's look at an example:

In [32]:
(let ((user-1 { first-name: "satoshi", last-name: "nakamoto", age: none })
      (user-2 { first-name: "john", last-name: "doe", age: (some 30) }))
    
    ;; Print ages of user-1 and user-2 
    (print (list (get age user-1) 
                 (get age user-2))))

(none (some 30))

Clarity language reference includes a few functions for easing manipulation for this type.



### (some value)
The `some` function constructs a `optional` type from the input value.

In [29]:
(some 1)

(some 1)

***
### none
Represents the _none_ option indicating no value for a given optional (analogous to a null value).

In [30]:
none

none

***
### (is-none value)
`is-none` tests a supplied option value, returning `true` if the option value is `(none)`,
and `false` if it is a `(some ...)`.

In [33]:
(define-constant satoshi { first-name: "satoshi", last-name: "nakamoto", ssn: none })

(print (if (is-none (get ssn satoshi))
        "Ghost"
        "Person"))

"Ghost"

***
### (is-some value)
`is-some` tests a supplied option value, returning `true` if the option value is `(some ...)`,
and `false` if it is a `none`.

***
### (default-to default-value option-value)

The `default-to` function attempts to 'unpack' the second argument: if the argument is
a `(some ...)` option, it returns the inner value of the option. If the second argument is a `(none)` value,
`default-to` it returns the value of `default-value`.

In [36]:
(let ((user-1 { first-name: "satoshi", last-name: "nakamoto", age: none })
      (user-2 { first-name: "john", last-name: "doe", age: (some 30) }))
    
    ;; Print ages of user-1 and user-2 
    (print (list (default-to 0 (get age user-1)) 
                 (default-to 0 (get age user-2)))))

(0 30)

Testing for nullability is often use for control flow, early returns and execution abortion. For that reason, Clarity language also includes some control flow related functions.

***
### (unwrap! option-input thrown-value)

The `unwrap!` function attempts to 'unpack' the first argument: if the argument is
an option type, and the argument is a `(some ...)` option, `unwrap!` returns the inner value of the
option. If the supplied argument is a `(none)` value,
`unwrap!` _returns_ `thrown-value` from the current function and exits the current control-flow.

In [19]:
(let ((user { first-name: "satoshi", last-name: "nakamoto", age: (some 30) }))
    (unwrap! (get age user) "Error - age must be provided")
    (print "Good to go"))


"Good to go"

***
### (try! option-input)
The `try!` function attempts to 'unpack' the first argument: if the argument is
an option type, and the argument is a `(some ...)` option, `try!` returns the inner value of the
option. If the supplied argument is a `none` value,
`try!` _returns_ `none` value from the current function and exits the current control-flow.

In [21]:
(let ((user { first-name: "satoshi", last-name: "nakamoto", age: (some 30) }))
    (print (try! (get age user))))

30

## 6) The response type and its associated functions

Similarly to the type `option`, Clarity introduces the type `response`, used for returning and propagting errors.
Response types have two variants, `(ok ...)`, representing success and containing a value, and `(err ...)`, representing error and containing an error value.

Let's look at an example:

In [38]:
(let ((money-transfer-1 (ok 1000))
      (money-transfer-2 (err "Failed: not enough funds")))
    
    ;; Print ages of user-1 and user-2 
    (print (list money-transfer-1 
                 money-transfer-2)))

((ok 1000) (err "Failed: not enough funds"))

Clarity language reference includes a few functions for easing manipulation for this type.

### (ok value)
The `ok` function constructs a response type from the input value. Use `ok` for
creating return values in public functions. An _ok_ value indicates that any database changes during
the processing of the function should materialize.

In [25]:
(ok 1)

(ok 1)

***
### (err value)
The `err` function constructs a response type from the input value. Use `err` for
creating return values in public functions. An _err_ value indicates that any database changes during
the processing of the function should be rolled back.

In [26]:
(err 1)

(err 1)

***
### (is-ok value)
`is-ok` tests a supplied response value, returning `true` if the response was `ok`,
and `false` if it was an `err`.

In [32]:
(print (list 
    (is-ok (ok 1)) 
    (is-ok (err 1))))

(true false)

***
### (is-err value)
`is-err` tests a supplied response value, returning `true` if the response was an `err`,
and `false` if it was an `ok`.

In [34]:
(print (list 
    (is-err (ok 1)) 
    (is-err (err 1))))

(false true)

Testing for responses is often use for control flow, early returns and execution abortion. For that reason, Clarity language also includes some control flow related functions.

### (try! response-input)
The `try!` function attempts to 'unpack' the first argument: if the argument is a response type, and the argument is an `(ok ...)` response, `try!` returns
 the inner value of the `ok`. If the supplied argument is an `(err ...)`,
`try!` _returns_ the `(err ...)` value from the current function and exits the current control-flow.

In [45]:
(define-private (make-transfer (successful bool))
    (if successful 
        (ok 1000) 
        (err "Transfer failed")))

(begin
    (try! (make-transfer true))
    (print "Transfer successful"))

→ .snippet-1 contract successfully stored. Use (contract-call? ...) for invoking the public functions:


***
### (unwrap! response-input thrown-value)

The `unwrap!` function attempts to 'unpack' the first argument: if the argument is a response type, and the argument is an `(ok ...)` response, `unwrap!` returns the inner value of the `ok`. If the supplied argument is an `(err ...)`,
`unwrap!` _returns_ `thrown-value` from the current function and exits the current control-flow.

In [57]:
(define-private (make-transfer (successful bool))
    (if successful 
        (ok 1000) 
        (err "Transfer failed")))

(begin
    (unwrap! (make-transfer false) (err "Execution halted"))
    (print "Transfer successful"))

Error: [1;31mRuntime Error: ShortReturn(ExpectedValue(Response(ResponseData { committed: false, data: Sequence(String("Execution halted")) })))[0m

You would use `try!` if you want to throw and cascade an error, and `unwrap!` if you want to override the error being thrown by the expression.

***
### (unwrap-err! response-input thrown-value)
The `unwrap-err!` function attempts to 'unpack' the first argument: if the argument
is an `(err ...)` response, `unwrap-err!` returns the inner value of the `err`.
If the supplied argument is an `(ok ...)` value,
`unwrap-err!` _returns_ `thrown-value` from the current function and exits the current control-flow.

In [61]:
(define-private (make-transfer (successful bool))
    (if successful 
        (ok 1000) 
        (err "Transfer failed")))

(begin
    (unwrap-err! (make-transfer false) (err "make-transfer should have failed but did succeed"))
    (print "Transfer not successful - as expected"))

→ .snippet-5 contract successfully stored. Use (contract-call? ...) for invoking the public functions:


***
### (unwrap-panic response-input)

The `unwrap` function attempts to 'unpack' its argument: if the argument is
an option type, and the argument is a `(some ...)` option, this function returns the inner value of the
option. If the argument is a response type, and the argument is an `(ok ...)` response, it returns
 the inner value of the `ok`. If the supplied argument is either an `(err ...)` or a `(none)` value,
`unwrap` throws a runtime error, aborting any further processing of the current transaction.

In [58]:
(unwrap-panic (ok 1))

1

***
### (unwrap-err-panic response-input)
The `unwrap-err` function attempts to 'unpack' the first argument: if the argument
is an `(err ...)` response, `unwrap` returns the inner value of the `err`.
If the supplied argument is an `(ok ...)` value,
`unwrap-err` throws a runtime error, aborting any further processing of the current transaction.

In [83]:
(unwrap-err-panic (err 1))

1